<a href="https://colab.research.google.com/github/ZahraArshia/Word2Image/blob/main/Word2Img_GoogleImagesSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google_images_search

In [ ]:
from google_images_search import GoogleImagesSearch
import pandas as pd
import requests
import json

def get_image_url(search_query, index=0):
    # Setting up Google Images Search API credentials
    gis = GoogleImagesSearch('API Key', 'Project ID')
    # Configure the search query
    search_params = {
        'q': search_query,
        'num': 1,  # Number of images to retrieve (change as needed)
        'safe': 'high',  # Filter explicit content: high, medium, off
    }

    try:
        # Perform the search
        gis.search(search_params)

        # Retrieve the first image URL from the search results
        if gis.results():
            return gis.results()[0].url
    except:

        url = "https://google.serper.dev/images"

        payload = json.dumps({
            "q": search_query,
            "num": 1,
            "gl": "iq",
            "hl": "ar" # setting language
            })

        api_keys = [
            # several API keys to support all dataset records by free plan
        ]

        headers = {
            'X-API-KEY': api_keys[index],
            'Content-Type': 'application/json'
        }

        response = requests.request("POST", url, headers=headers, data=payload)
        try:
            image_url = response.json()['images'][0]['imageUrl']
            return image_url
        except:
            return None

    return None



In [ ]:
# initial var
image_url_list = []
start_index = 0
api_key_index = 0
max_api_index = 7
api_limit = 2500

In [ ]:
# for continue from last word find image url read from excel
data_path = '/Arabic_words_output1.xlsx'
df = pd.read_excel(data_path)
image_url_list = list(df["Column7"])
for value in image_url_list:
  if pd.isna(value):
    start_index = image_url_list.index(value)
    print("first index is Nan item: ", start_index)
    break

In [ ]:
# usage
data_path = '/Arabic_Words_Uniqe.xlsx'
df = pd.read_excel(data_path)
counter = 0

# find index of api key must send request
for i in range(max_api_index):
  if start_index < ((i+1) * api_limit):
    api_key_index = i

# if image_url_empty make it with None value and quantity = length of words
if len(image_url_list) == 0:
  image_url_list = [None] * len(df["Column4"])

print("start index:", start_index)
print("api key index:", api_key_index)

for word in df["Column4"]:
    print(counter, end="-")
    counter += 1
    if counter < start_index:
      continue
    search_word = word
    image_url = get_image_url(search_word, api_key_index)
    if image_url:
        image_url_list[counter]=image_url
        print(f"Image URL for '{search_word}': {image_url}")
    else:
      image_url_list[counter]=""
      if api_key_index < max_api_index - 1:
        api_key_index += 1
        counter -= 1
      print(f"No image found for '{search_word}'")

In [ ]:
image_url_list[-1]

In [ ]:
df["Column7"] = image_url_list
df.to_excel('output.xlsx', index=False)